# Evolutionary Algorithm Visualization

This notebook visualizes the progress and results of the Evolutionary Algorithm for ODE discovery.
We will run the EA for a few generations and plot:
1. Fitness evolution (MSE trend) for each island.
2. Trajectories of the best discovered model vs. the ground truth.

In [ ]:
import sys
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure code is in path
sys.path.append(os.getcwd())

from data.generate_data import WarfarinTMDDDataset
from evolution.llm_interface import LLMClient
from evolution.models import Island, Individual
from evolution.algorithm import EvolutionEngine
from evolution.models import integrate_euler

# Setup plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
print("--- Setting up Data ---")

# 1. Setup Data (Same as previous experiments)
true_params = {
    'CL': 0.1, 'V1': 1.0, 'Q': 0.5, 'V2': 2.0,
    'kon': 0.01, 'koff': 0.005, 'kdeg': 0.001, 'ksyn': 0.002, 
    'initial_state': [10.0, 0.0, 0.0, 5.0],
    't_start': 0, 't_end': 50, 'num_points': 50
}
generator = WarfarinTMDDDataset(generation_parameters=true_params)
gt_data = generator.generate()
time_points = generator.get_time_points()
obs_indices = [0, 2] # x1, x3

observed_data = gt_data[:, obs_indices]

In [ ]:
print("--- Setting up EA ---")

# 2. Setup Islands
# We will run small populations for visualization speed
islands = [
    Island(num_vars=2, capacity=3),
    Island(num_vars=3, capacity=3),
    Island(num_vars=4, capacity=3)
]

# 3. Initialize Population
llm = LLMClient()
problem_desc = "Pharmacokinetic/Pharmacodynamic system of Warfarin. Central and Peripheral compartments. Drug binds to a target. Clearance from Central."

print("Initializing Populations (This may take a minute)... ")
for island in islands:
    print(f"  Initializing Island (Dim {island.num_vars})...")
    for _ in range(island.capacity):
        code = llm.generate_initial_model(problem_desc, island.num_vars)
        if code:
            ind = Individual(code)
            if ind.compile():
                ind.evaluate(observed_data, obs_indices, time_points)
                island.add_individual(ind)
                print(".", end="", flush=True)
        else:
            print("x", end="", flush=True)
    print()

In [ ]:
# 4. Run Evolution Loop
data_pkg = {'observed_data': observed_data}
engine = EvolutionEngine(islands, llm, data_pkg, obs_indices, time_points, n_iterations=20)

num_generations = 5 # Run for 5 generations
history = []

print("Starting Evolution...")
for g in range(num_generations):
    gen_stats = engine.run_generation()
    history.extend(gen_stats)

In [ ]:
# 5. Plot Fitness History
df_history = pd.DataFrame(history)

plt.figure(figsize=(10, 6))
sns.lineplot(data=df_history, x='generation', y='best_fitness', hue='num_vars', marker='o', palette='viridis')
plt.title('Best Fitness (MSE) by Island over Generations')
plt.xlabel('Generation')
plt.ylabel('MSE Loss (Log Scale)')
plt.yscale('log')
plt.legend(title='Num Vars')
plt.grid(True, which="both", ls="--")
plt.show()

In [ ]:
# 6. Visualize Best Model Trajectory
best_overall = None
for island in islands:
    b = island.get_best()
    if b:
        if best_overall is None or b.fitness < best_overall.fitness:
            best_overall = b

print(f"Best Overall Fitness: {best_overall.fitness:.4f}")
print(f"Best Model Dimension: {best_overall.model.num_vars}")

# Simulate Best Model
# Re-integrate to get full trajectory
# Note: reconstruction of x0 logic must be identical to 'evaluate'
x0 = torch.zeros(best_overall.model.num_vars)
for i, obs_idx in enumerate(obs_indices):
    if obs_idx < best_overall.model.num_vars:
        x0[obs_idx] = observed_data[0, i]
x0[x0 == 0] = 0.1

with torch.no_grad():
    pred_traj = integrate_euler(best_overall.model, x0, time_points).numpy()

# Plotting
plt.figure(figsize=(12, 6))

# Plot Ground Truth Observations
colors = ['blue', 'green']
labels = ['Observed x1 (Central)', 'Observed x3 (Target)']

for i, obs_idx in enumerate(obs_indices):
    plt.scatter(time_points, observed_data[:, i], color=colors[i], label=f'{labels[i]} (Data)', alpha=0.6)

# Plot Model Predictions
# We plot the indices that theoretically match.
# Assumption: Model indices 0 and 1 match obs indices 0 and 2 if dim=2? 
# Or we assume naive mapping: index 0 -> obs 0, index 2 -> obs 1 if available?
# In the 'evaluate' method, we matched by explicit index.
# for i, obs_idx in enumerate(obs_indices):
#    if obs_idx in valid_indices...

for i, obs_idx in enumerate(obs_indices):
    if obs_idx < pred_traj.shape[1]:
        plt.plot(time_points, pred_traj[:, obs_idx], color=colors[i], linestyle='--', label=f'Model x{obs_idx}')

# Plot Hidden if available (e.g. x2)
if pred_traj.shape[1] > 1 and 1 not in obs_indices:
    plt.plot(time_points, pred_traj[:, 1], color='red', linestyle=':', label='Model x1 (Hidden?)', alpha=0.5)

plt.title('Best Discovered Model vs. Ground Truth Data')
plt.xlabel('Time')
plt.ylabel('Concentration')
plt.legend()
plt.show()

In [ ]:
print("Best Discovered Code:")
print(best_overall.code)